# 模型层

# 聚合查询

只要是跟数据库线管的模块基本上都在**django.db.models**里面，如果没有就是在**django.db**中

聚合查询通常情况下都是配合分组一起使用。

模块导入：
`from django.db.models import Max, Min, Sum, Count, Avg`


1. 所有书的评价价格

```
res = models.Book.objects.aggregate(Avg('price'))
```

2. 上述方法一次性使用

```
res = models.Book.objects.aggregate(Avg('price'), Max('prince'))
```

# 分组查询

1. 统计每一本书的作者个数

```
res = models.Book.objects.annotate()  # models后面点什么,就是按什么分组,此时按Book分组

res = models.Book.objects.annotate(author_num=Count('authors')).values('title', 'author_num')

# author_num 是自己定义的字段,用来存储统计出来的额每本书对应的作者个数
```

2. 统计每个出版社卖的最便宜的书的价格

```
res = models.Publish.objects.annotate(min_price=Min('book__price')).values('name', 'min_price')
```

3. 统计不止一个作者的图书

```
# 1.先按照图书分组,求每一本书对应作者个数
# 2.过滤出不止一个作者的图书

res = models.Book.objects.annotate(author_count=Count('authors')).filter(author_count__gt=1).values('name', 'author_count')
```

**只要你的orm语句得出的结果还是一个queryset对象,那么它就可以继续无限制的点queryset对象封装的方法**

按照指定的字段分组:

`models.Book.objects.valus('price').annotate()`

# F与Q查询

## F

1. 查询卖出数大于库存数的书籍

```
F 查询,能够帮助你直接获取到表中的某个字段对应的数据

from django.db.models import F
res = models.Book.objects.filter(maichu__gt=F('kucun'))
```

2. 将所有书记的价格提升500块

```
models.Book.objects.update(price=F('price') + 500)
```

3. 将所有书的名称后面加上爆款两个字

```
在操作字符类型的数据时,F不能够直接做到字符串的拼接

from django.db.models.functions import Concat
from django.db.models import Value
models.Book.objects.update(title=Concat(F('title'), Value('爆款')))
```

## Q

1. 查询卖出数大于100或者价格小于600的书籍

```
from django.db.models import Q
res = models.Book.objects.filter(Q(maichu__gt=100)|Q(price_lt=600))  # | 是 or关系,使用,分割还是and关系

res = models.Book.objects.filter(~Q(maichu__gt=100)|Q(price_lt=600))  # ~ 是 not关系
```

Q 的高阶用法,能够将查询条件的左边也变成字符串的形式

```
q = Q()
q.children.append(('maichu_gt', 100))
q.children.append(('price_lt', 600))

res = models.Book.objects.filter(q)  # 默认是and关系

修改链接关系:
q.connector = 'or'
```